In [ ]:
%cd ../

In [2]:
from tvDatafeed import TvDatafeed, Interval
from dotenv import load_dotenv
import os
import pandas as pd
import pyutil
import backtrader as bt
import numpy as np
import talib as ta
import json
import pickle
import datetime
from tabulate import tabulate
from prettytable import PrettyTable
import tenxsqueeze as txs
import inspect
import seaborn as sns
from tqdm import tqdm

load_dotenv()

sp500_5m = pd.read_pickle("data/sp500_5m_clean.pkl")
good_tickers = pd.Series({
    ticker: data.open.isna().sum() for ticker, data in sp500_5m.items()
}).pipe(lambda s: s[s < 15])
good_tickers
sp500_5m = {ticker: v.ffill() for ticker, v in sp500_5m.items() if ticker in good_tickers.index}

In [3]:
def delay_timestamp(inp, delay=pd.Timedelta(minutes=5)):
    return inp.reset_index().assign(datetime=inp.index + delay).set_index("datetime")

def add_candlesticks_to_cerebro(cerebro, candlesticks: dict):
    for ticker, data in list(candlesticks.items())[:]:
        candlestick_feed = bt.feeds.PandasData(dataname=delay_timestamp(data), timeframe=bt.TimeFrame.Minutes, compression=5)
        cerebro.adddata(candlestick_feed, name=f"{ticker}_5m")
        # cerebro.replaydata(candlestick_feed, timeframe=bt.TimeFrame.Minutes, compression=15, name=f"{ticker}_15m")
        # cerebro.replaydata(candlestick_feed, timeframe=bt.TimeFrame.Minutes, compression=30, name=f"{ticker}_30m")

cerebro = bt.Cerebro()

add_candlesticks_to_cerebro(cerebro, sp500_5m)

cerebro.addstrategy(txs.strategies.MTFB3)
cerebro.broker.setcash(100000.0)
cerebro.broker.setcommission(commission=0.0006)

ret = cerebro.run(stdstats=False, objcache=False, logging=False)

In [5]:
order_info = pd.DataFrame(ret[0].order_info)
#order_info.to_pickle("order_info.pkl")
order_info

,ticker,dt,filled_price,size,value,comm,pnl,pnl%,len,kind
0,ZTS,2023-06-12 10:00:00,163.460000,-1,-163.460000,0.098076,0.000000,0.000000,98,Entry
1,ZTS,2023-06-12 10:05:00,163.847820,1,163.847820,0.098309,-0.584204,-0.356553,99,SL
2,FI,2023-06-12 10:05:00,117.190000,1,117.190000,0.070314,0.000000,0.000000,99,Entry
3,TJX,2023-06-12 10:05:00,79.530000,1,79.530000,0.047718,0.000000,0.000000,99,Entry
4,FI,2023-06-12 10:10:00,117.020000,1,117.020000,0.070212,0.000000,0.000000,100,Entry
...,...,...,...,...,...,...,...,...,...,...
5436,ETSY,2023-09-11 13:40:00,65.653821,3,196.961463,0.118177,-0.754954,-0.383301,4941,SL
5437,VTR,2023-09-11 14:55:00,42.950000,1,42.950000,0.025770,0.000000,0.000000,4956,Entry
5438,VTR,2023-09-11 15:00:00,43.012825,-2,-86.025649,0.051615,-0.040337,-0.046889,4957,TP
5439,VTR,2023-09-11 15:00:00,42.960000,1,42.960000,0.025776,0.000000,0.000000,4957,Entry


In [6]:
order_info[order_info.ticker == "ALK"]

,ticker,dt,filled_price,size,value,comm,pnl,pnl%,len,kind
430,ALK,2023-06-15 12:20:00,49.970000,-1,-49.970000,0.029982,0.000000,0.000000,360,Entry
456,ALK,2023-06-15 13:35:00,50.058000,1,50.058000,0.030035,-0.148017,-0.295690,375,SL
2066,ALK,2023-07-11 11:55:00,55.970000,1,55.970000,0.033582,0.000000,0.000000,1567,Entry
2069,ALK,2023-07-11 12:00:00,55.960000,1,55.960000,0.033576,0.000000,0.000000,1568,Entry
2072,ALK,2023-07-11 12:05:00,56.050000,1,56.050000,0.033630,0.000000,0.000000,1569,Entry
2074,ALK,2023-07-11 12:10:00,56.045000,-3,-168.135000,0.100881,-0.046669,-0.027757,1570,SL
3225,ALK,2023-07-26 12:35:00,47.360000,-1,-47.360000,0.028416,0.000000,0.000000,2433,Entry
3226,ALK,2023-07-26 12:45:00,47.475992,1,47.475992,0.028486,-0.273682,-0.576463,2435,SL
3539,ALK,2023-08-02 11:15:00,47.780000,-1,-47.780000,0.028668,0.000000,0.000000,2807,Entry
3540,ALK,2023-08-02 11:20:00,47.730000,-1,-47.730000,0.028638,0.000000,0.000000,2808,Entry


In [7]:
order_info[(order_info.kind == "SL") | (order_info.kind == "TP")].sort_values("pnl%", ascending=False).head(20)

,ticker,dt,filled_price,size,value,comm,pnl,pnl%,len,kind
138,ZION,2023-06-13 09:30:00,28.670000,8,229.360000,0.137616,11.542673,5.032557,170,TP
1021,ENPH,2023-06-23 09:30:00,159.760000,9,1437.840000,0.862704,24.349670,1.693490,716,TP
4167,TFC,2023-08-15 09:30:00,30.020000,4,120.080000,0.072048,1.613811,1.343947,3488,TP
480,ENPH,2023-06-16 09:30:00,183.200000,-4,-732.800000,0.439680,6.254918,0.853564,404,TP
397,INTC,2023-06-15 10:25:00,36.020188,-1,-36.020188,0.021612,0.302171,0.838893,337,TP
920,ADBE,2023-06-22 10:55:00,474.974900,3,1424.924699,0.854955,11.647373,0.817403,655,TP
3915,SEE,2023-08-09 10:10:00,39.630000,4,158.520000,0.095112,1.193945,0.753183,3184,TP
4578,FSLR,2023-08-18 11:30:00,180.889125,7,1266.223877,0.759734,8.936333,0.705747,3746,TP
4659,GIS,2023-08-21 09:30:00,69.690000,3,209.070000,0.125442,1.458090,0.697417,3800,TP
610,AVGO,2023-06-20 10:25:00,864.737964,2,1729.475929,1.037686,11.090796,0.641281,493,TP


In [9]:
order_info.pnl.sum()

-1118.9475291975095